## KMZ Rasters to GPGK Polygons
Read raw radio propagation output, convert all raster bands to one dissolved polygon, export as GPKG.

#### Set-up

In [2]:
import geopandas as gpd
import pandas as pd
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape
import os
import glob

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [11]:
# Select country and folder within which radio propagation rasters are output
country = 'togo'
itu_table = 'fmtv'

Code source: https://gis.stackexchange.com/questions/187877/how-to-polygonize-raster-to-shapely-polygons

In [3]:
mask = None

def raster_to_polygon(in_path, out_path):
        with rasterio.open(in_path) as src:
                image = src.read(1) # first band
                transform = src.transform # transform so that pixels have a reference system
                results = (
                        {'properties': {'raster_val': v}, 'geometry': s}
                        for i, (s, v) in enumerate(
                        shapes(image, mask=mask, transform=src.transform))
                        
                        # Ignore radio bands that have a value of 0 (I assume this means no signal)
                        if v != 0
                )
        
                # Create list of geoms
                geoms = list(results)

                # Convert to geodataframe
                gdf = gpd.GeoDataFrame.from_features(geoms)

                # Dissolve all polygons into a single geometry
                dissolved = gdf.dissolve()

                # Export
                dissolved.to_file(out_path)
        

In [9]:

def raster_to_polygon(in_path, out_path):
    with rasterio.open(in_path) as src:
        transform = src.transform  # Reference system
        
        # Read all bands and store valid polygons
        geoms = []
        for band in range(1, src.count + 1):  # Loop through all bands
            image = src.read(band)
            
            results = (
                {'properties': {'raster_val': v}, 'geometry': shape(s)}
                for s, v in shapes(image, mask=None, transform=transform)
                if v != 0  # Ignore areas with no signal
            )
            
            geoms.extend(results)
        
        # Convert to GeoDataFrame
        gdf = gpd.GeoDataFrame.from_features(geoms)
        
        # Dissolve all polygons into a single geometry
        dissolved = gdf.dissolve()
        
        # Ensure output is a MultiPolygon
        dissolved.geometry = dissolved.geometry.apply(lambda geom: MultiPolygon([geom]) if geom.geom_type == "Polygon" else geom)
        
        # Export
        dissolved.to_file(out_path, driver="GPKG")
        
    print(f"Polygon saved to {out_path}")

In [ ]:
for file in glob.glob(f'output/raw/{country}/{itu_table}/*'):

    # Define output path, files must be in kmz format
    outpath = file.replace('raw', 'gpkg').replace('.kmz', '.gpkg')
    print("Output: ", outpath)

    # Read & polygonize raster, and export polygon
    raster_to_polygon(file, outpath)

Output:  output/gpkg/togo/fmtv/2025-03-23_102215_RADIO RTDS_ANEHO.gpkg
Polygon saved to output/gpkg/togo/fmtv/2025-03-23_102215_RADIO RTDS_ANEHO.gpkg
Output:  output/gpkg/togo/fmtv/2025-03-23_102445_RADIO OCEAN_ANEHO.gpkg
Polygon saved to output/gpkg/togo/fmtv/2025-03-23_102445_RADIO OCEAN_ANEHO.gpkg
Output:  output/gpkg/togo/fmtv/2025-03-23_100900_RADIO RFI AGOU_AGOU.gpkg
Polygon saved to output/gpkg/togo/fmtv/2025-03-23_100900_RADIO RFI AGOU_AGOU.gpkg
Output:  output/gpkg/togo/fmtv/2025-03-23_101317_FIRMAMENT_AGOU GADZEPE.gpkg
Polygon saved to output/gpkg/togo/fmtv/2025-03-23_101317_FIRMAMENT_AGOU GADZEPE.gpkg
Output:  output/gpkg/togo/fmtv/2025-03-23_101755_RADIO LOME ALEJO_ALEJO.gpkg
Polygon saved to output/gpkg/togo/fmtv/2025-03-23_101755_RADIO LOME ALEJO_ALEJO.gpkg
Output:  output/gpkg/togo/fmtv/2025-03-23_103351_RADIO LOME_ATAKPAME.gpkg
Polygon saved to output/gpkg/togo/fmtv/2025-03-23_103351_RADIO LOME_ATAKPAME.gpkg
Output:  output/gpkg/togo/fmtv/2025-03-23_101057_RADIO EPHATHA